In [ ]:
import sys
import pandas as pd
import scresonators.measurement.handy as hy
import os 
import scipy.special as sp
import numpy as np 

eps_sil = 11.7 # silicon 
eps_sapp = 9.3 # sapphire 
rat_sil = 1.77 
rat_sapp = 2.39

# for participation ratio
eps_c = 10
delta = 0.003

def kp(k): return np.sqrt(1-k**2)
def Kp(k): return sp.ellipk(kp(k))
def K(k): return sp.ellipk(k)

def psm(a, b, eps):
    return eps**2 / eps_c/(eps+1) / (2 * (1-a/b)*Kp(a/b)*K(a/b)) * delta/a * (np.log(4*(1-a/b)/(1+a/b)) - a/b * np.log(a/b) / (1+a/b) + 1 - np.log(delta/a))

def psm2(pin, gap, eps):
    return psm(pin/2, pin/2+gap, eps)

pth_base ='/Users/sph/Library/CloudStorage/GoogleDrive-circuitqed@gmail.com/'
cloud_folder = 'My Drive/Projects/Materials/'

img_path = os.path.join(pth_base, cloud_folder, 'Outputs/Resonators/')
yml_pth = os.path.join(pth_base, cloud_folder, 'Outputs/Resonators/Text')
from notion_utils import get_notion_client, get_notion_db


n2p = get_notion_client()
lnk = get_notion_db()

## Sample params

In [11]:
lnk.columns

Index(['Sputterer ID', 'Gap (um)', 'Q_Hi', 'Target Freq. (GHz)', 'P_ms',
       'File', 'Qother (1e6)', 'Measurement', 'Post clean', 'Freq (GHz)',
       'Package', 'Qother err (1e6)', 'Qc err (1e6)', 'Samples',
       'Thickness (nm)', 'QTLS0 (1e6)', 'Wafer', 'Surface Prep',
       'Fridge Cool Down', 'Depo', 'QTLS0 err (1e6)', 'Qc (1e6)', 'Date',
       'Metal', 'Substrate', 'Pattern', 'Pin (um)', 'Growth temp (C)',
       'Separation (um)', 'Name', 'PageID', 'Row_Hash'],
      dtype='object')

## Create a data frame with new data 

In [8]:
# Now, make a df that has the same columns as the notion DB
cols=['freqs', 'pitch', 'qc', 'qc_err', 'qother', 'qtls0', 'qtls0_err', 'qother_err', 'target_freq']
yml_info=hy.load(os.path.join(yml_pth,p))
new_df = pd.DataFrame({col: yml_info[col] for col in cols})

time_dict =  {'start': date_meas, 'end': None, 'time_zone': None}
new_df['Date'] = new_df.apply(lambda row: time_dict, axis=1)
new_df['Metal'] = metal
new_df['Substrate'] = substrate
new_df['Depo']=depo
new_df['Package']=package
new_df['Measurement']=meas
new_df['Notes'] = notes
new_df['PageID']=''
new_df['Row_Hash']=''
new_df['Fridge Cool Down']=''
new_df['Name']=name
new_df['Samples']=''
new_df['Pattern']=Pattern

new_df.rename(columns={'pitch': 'Gap (um)', 'qtls0': 'QTLS0 (1e6)', 'qother': 'Qother (1e6)', 'qother_err':'Qother err (1e6)', 'qc':'Qc (1e6)', 'qc_err':'Qc err (1e6)','qtls0_err':'QTLS0 err (1e6)', 'target_freq':'Target Freq. (GHz)', 'freqs':'Freq (GHz)' }, inplace=True)
round_cols = ['QTLS0 (1e6)', 'Qother (1e6)', 'Qc (1e6)', 'Qc err (1e6)', 'QTLS0 err (1e6)', 'Qother err (1e6)']
new_df[round_cols] = new_df[round_cols] / 1e6
new_df['Freq (GHz)'] = new_df['Freq (GHz)'] / 1e9

new_df[round_cols] = new_df[round_cols].round(4)
f_cols = ['Freq (GHz)', 'Target Freq. (GHz)']
new_df[f_cols] = new_df[f_cols].round(7)

new_df['Pin (um)'] = new_df.apply(
    lambda row: row['Gap (um)'] * rat_sapp if row['Substrate'] == 'Sapphire' else row['Gap (um)'] * rat_sil,
    axis=1
)

new_df['Pin (um)'] = new_df.apply(
    lambda row: row['Gap (um)'] if row['Gap (um)'] in [3, 30] else row['Pin (um)'],
    axis=1
)

new_df['P_ms'] = new_df.apply(
    lambda row: psm2(row['Pin (um)'], row['Gap (um)'], eps_sapp if row['Substrate'] == 'Sapphire' else eps_sil),
    axis=1
)

## Load notion database as dataframe and add new data to it 

In [9]:
notion_df = n2p.from_notion_DB_to_dataframe(lnk)
combined_df = pd.concat([notion_df, new_df], ignore_index=True)
#print(combined_df)

## Send updated df to notion

In [10]:
n2p.update_notion_DB_from_dataframe(lnk, combined_df)

In [ ]:
notion_df = n2p.from_notion_DB_to_dataframe(lnk)

# Add P_ms data

In [3]:
import scipy.special as sp
import numpy as np 

eps_sil = 11.7 # silicon 
eps_sapp = 9.3 # sapphire 
rat_sil = 1.77 
rat_sapp = 2.39

eps_c = 10
delta = 0.003

def kp(k): return np.sqrt(1-k**2)
def Kp(k): return sp.ellipk(kp(k))
def K(k): return sp.ellipk(k)

def psm(a, b, eps):
    return eps**2 / eps_c/(eps+1) / (2 * (1-a/b)*Kp(a/b)*K(a/b)) * delta/a * (np.log(4*(1-a/b)/(1+a/b)) - a/b * np.log(a/b) / (1+a/b) + 1 - np.log(delta/a))

def psm2(pin, gap, eps):
    return psm(pin/2, pin/2+gap, eps)

In [ ]:
# Fill in values for 'Pin (um)' in notion_df based on the substrate
notion_df['Pin (um)'] = notion_df.apply(
    lambda row: row['Gap (um)'] * rat_sapp if row['Substrate'] == 'Sapphire' else row['Gap (um)'] * rat_sil,
    axis=1
)

notion_df['Pin (um)'] = notion_df.apply(
    lambda row: row['Gap (um)'] if row['Gap (um)'] in [3, 30] else row['Pin (um)'],
    axis=1
)

In [9]:
notion_df['P_ms'] = notion_df.apply(
    lambda row: psm2(row['Pin (um)'], row['Gap (um)'], eps_sapp if row['Substrate'] == 'Sapphire' else eps_sil),
    axis=1
)

In [19]:
notion_df['Pattern'] = notion_df['Name'].apply(
    lambda x: 'Houck_Sapp' if x == 'Sil1_SQ' else ('QcAdj_Sil' if x == 'Sil7-R1' else 'Houck_Sil')
)

In [20]:
n2p.update_notion_DB_from_dataframe(lnk, notion_df)

# Add new column based on yml files 

In [2]:
# Import the function
import sys
sys.path.append('..')
from get_dataframe_with_q_hi import get_dataframe_with_q_hi

# Get the dataframe with Q_Hi column
df_with_q_hi = get_dataframe_with_q_hi()

# Now you have the complete dataframe with Q_Hi values!
print(df_with_q_hi.shape)  # (119, 32)
print(df_with_q_hi['Q_Hi'].head())  # Shows Q_Hi values

Loading Notion database...
Current dataframe shape: (119, 32)
Looking for YAML files in: /Users/sph/Library/CloudStorage/GoogleDrive-circuitqed@gmail.com/My Drive/Projects/Materials/Outputs/Resonators/Text
Processed 1/119 rows... (1 matches so far)
Processed 21/119 rows... (21 matches so far)
Processed 41/119 rows... (41 matches so far)
Processed 61/119 rows... (61 matches so far)
Processed 81/119 rows... (81 matches so far)
Processed 101/119 rows... (101 matches so far)

Added Q_Hi column with 119 successful matches out of 119 total rows
Final dataframe shape: (119, 32)

Sample Q_Hi values:
                                       File  Gap (um)      Q_Hi
0  250728-ResSilTa-Aggron-JPAsilta8_jpa.yml        35  0.242979
1  250728-ResSilTa-Aggron-JPAsilta8_jpa.yml        30  1.433132
2  250728-ResSilTa-Aggron-JPAsilta8_jpa.yml        16  1.320735
3  250728-ResSilTa-Aggron-JPAsilta8_jpa.yml        10  1.120163
4  250728-ResSilTa-Aggron-JPAsilta8_jpa.yml         6  0.695672
(119, 32)
0    0.

In [3]:
from notion_utils import send_notion_db

send_notion_db(df_with_q_hi)


### They need to have all the same rows and columns

In [76]:
#print(notion_df.columns)
#print(new_df.columns)

common_columns = notion_df.columns.intersection(new_df.columns)
print("Common columns:", common_columns)

notion_only_columns = notion_df.columns.difference(new_df.columns)
print("Columns only in notion_df:", notion_only_columns)

new_df_only_columns = new_df.columns.difference(notion_df.columns)
print("Columns only in new_df:", new_df_only_columns)

Common columns: Index(['Gap (um)', 'Target Freq. (GHz)', 'Qother (1e6)', 'Measurement',
       'Notes', 'Freq (GHz)', 'Package', 'Qother err (1e6)', 'Qc err (1e6)',
       'Samples', 'QTLS0 (1e6)', 'Fridge Cool Down', 'Depo', 'QTLS0 err (1e6)',
       'Qc (1e6)', 'Date', 'Metal', 'Substrate', 'Name', 'PageID', 'Row_Hash'],
      dtype='object')
Columns only in notion_df: Index([], dtype='object')
Columns only in new_df: Index([], dtype='object')
